In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import os
from config import NDVI_DATA_URL_WSL
from netCDF4 import Dataset

In [2]:
for year in range(1981, datetime.today().year + 1):
    print(year)
    days_count = 0
    if not str(year) in os.listdir(NDVI_DATA_URL_WSL):
        os.mkdir(f"{NDVI_DATA_URL_WSL}/{year}")
    r = requests.get(f"https://www.ncei.noaa.gov/data/land-normalized-difference-vegetation-index/access/{year}/")
    soup = BeautifulSoup(r.content, 'html.parser')
    tds = soup.find_all("td")
    files = []
    for td in tds:
        if td.text[-3:] == ".nc":
            files.append(td.text)
    for file in files:
        date = file.split("_")[4]
        file_req = requests.get(f"https://www.ncei.noaa.gov/data/land-normalized-difference-vegetation-index/access/{year}/{file}")
        file_req.raise_for_status()
        with open(f"{NDVI_DATA_URL_WSL}/{year}/{date}.nc", "wb") as f:
            f.write(file_req.content)
        days_count += 1
    print(days_count)
    break

1981


OSError: [Errno 5] Input/output error

In [5]:
Dataset(f"{NDVI_DATA_URL}/1981/19810624.nc").variables["latitude"][:]

masked_array(data=[ 89.975  ,  89.925  ,  89.875  , ..., -89.875  ,
                   -89.925  , -89.97499],
             mask=False,
       fill_value=1e+20,
            dtype=float32)

In [10]:
Dataset(f"{NDVI_DATA_URL}/1981/out.nc").variables["latitude"][:]

masked_array(data=[50.975   , 50.925   , 50.874996, 50.824997, 50.774998,
                   50.725   , 50.675   , 50.624996, 50.574997, 50.524998,
                   50.475   , 50.425   , 50.374996, 50.324997, 50.274998,
                   50.225   , 50.175   , 50.124996, 50.074997, 50.024998,
                   49.975   , 49.925   , 49.874996, 49.824997, 49.774998,
                   49.725   , 49.675   , 49.624996, 49.574997, 49.524998,
                   49.475   , 49.425   , 49.374996, 49.324997, 49.274998,
                   49.225   , 49.175   , 49.124996, 49.074997, 49.024998,
                   48.975   , 48.925   , 48.874996, 48.824997, 48.774998,
                   48.725   , 48.675   , 48.624996, 48.574997, 48.524998,
                   48.475   , 48.425   , 48.374996, 48.324997, 48.274998,
                   48.225   , 48.175   , 48.124996, 48.074997, 48.024998,
                   47.975   , 47.925   , 47.874996, 47.824997, 47.774998,
                   47.725   , 47.675  